## This is HNA or Hybrid SFLA-CSO Network Aligner program, and implemented by Elham Mahdipour that is Ph.D. Candidate of Artificial Intelligence at Yazd University, Yazd, Iran. 
### All copyright reserved by the author.

You can run the program from first to end and see all the results, but since the population generation process can take a time depended on the node numbers of networks, you can use the populations stored in each program. We saved the populations as the pair of related species that you can see in the input folder and load them on program, such as "ce-sc-pop.pickle". Therefore, you must run follow steps: 1) Run all cells in "Load Graphs", "Swap Graphs", "Define Target Graph",and "Compute Score for create similarity matrix" sections. 2) If you will use saved population, please upload related pickle on root of jupyter notebook, then go "load population" cell and run them; else, please run "multi tasking for initial population", "multi processing for speed up", and "Initialization such as MeAlign" cells. 3) Run all remains cells to create results.

# Load Graphs 

In [1]:
import pickle
with open('mm-mm-weighted.pickle', 'rb') as f:
    mmlist = pickle.load(f)

In [2]:
with open('dm-dm-weighted.pickle', 'rb') as f:
    dmlist = pickle.load(f)

In [3]:
import networkx as nx
G1=nx.Graph()
for i in range(len(mmlist)):
    if G1.has_node(mmlist[i][0])==False:
        G1.add_node(mmlist[i][0])
    if G1.has_node(mmlist[i][1])==False:
        G1.add_node(mmlist[i][1])
    G1.add_edge(mmlist[i][0],mmlist[i][1],weight=mmlist[i][2])
G1

In [4]:
G1.number_of_nodes()

623

In [5]:
G1.number_of_edges()

1760

In [6]:
len(G1)

623

In [7]:
G2=nx.Graph()
for i in range(len(dmlist)):
    if G2.has_node(dmlist[i][0])==False:
        G2.add_node(dmlist[i][0])
    if G2.has_node(dmlist[i][1])==False:
        G2.add_node(dmlist[i][1])
    G2.add_edge(dmlist[i][0],dmlist[i][1],weight=dmlist[i][2])
G2

In [8]:
print('len G1:', len(G1))

print('len G2:',len(G2))

len G1: 623
len G2: 7361


# Swap graphs if len G1 > len G2

In [9]:
### Check and Swap if G1 > G2 ###
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

623
7361


In [10]:
ed1=G1.edges()

In [11]:
nd1=G1.nodes()

In [12]:
ed2=G2.edges()

In [13]:
len(ed2)

33691

In [14]:
nd2=G2.nodes()

In [15]:
len(ed1)

1760

In [16]:
len(nd1)

623

In [17]:
el1=list(ed1)
el2=list(ed2)

In [18]:
nd1=list(nd1)
nd2=list(nd2)

In [19]:
degG1 = [val for (node, val) in G1.degree()]


In [20]:
degG2 = [val for (node, val) in G2.degree()]

# Define Target Graph

In [21]:
G_target_na=nx.read_weighted_edgelist('dm-mm.evals')
G_target_na  

In [22]:
gtnode=list(G_target_na.nodes())
print(len(gtnode))
g_t_node=[]
for i in range(len(gtnode)):
    if (G1.has_node(gtnode[i])==False and G2.has_node(gtnode[i])==False):
        continue
    else:
        g_t_node.append(gtnode[i])

#print(gtnode)
print(len(g_t_node))

26167
6356


In [23]:
G_target=nx.Graph()
for i in range(len(g_t_node)):
    for j in range(len(g_t_node)):
        if G_target_na.has_node(g_t_node[i])==False:
            G_target.add_node(g_t_node[i])
        if G_target_na.has_node(g_t_node[j])==False:
            G_target.add_node(g_t_node[j])
        if G_target_na.has_edge(g_t_node[i],g_t_node[j]):
            a=list(G_target_na.get_edge_data(g_t_node[i],g_t_node[j]).values())
            #b=list(a[0][0].values())
            G_target.add_edge(g_t_node[i],g_t_node[j],weight=a[0])
G_target

In [24]:
len(G_target)

2331

In [25]:
G_target_na=G_target

In [26]:
G_target_na.edges()

EdgeView([('dm4970', 'mm8699'), ('dm4970', 'mm592'), ('dm4970', 'mm11243'), ('dm4970', 'mm2895'), ('dm4970', 'mm1347'), ('dm4970', 'mm9407'), ('dm4970', 'mm5183'), ('dm4970', 'mm5688'), ('dm4970', 'mm7088'), ('dm4970', 'mm1105'), ('dm4970', 'mm14063'), ('dm4970', 'mm1150'), ('dm4970', 'mm14346'), ('dm4970', 'mm6126'), ('dm4970', 'mm11666'), ('dm4970', 'mm8967'), ('mm8699', 'dm2741'), ('mm8699', 'dm601'), ('mm8699', 'dm4175'), ('mm8699', 'dm5604'), ('mm8699', 'dm3290'), ('mm8699', 'dm7122'), ('mm8699', 'dm1181'), ('mm8699', 'dm37'), ('mm8699', 'dm1918'), ('mm8699', 'dm801'), ('mm8699', 'dm3879'), ('mm8699', 'dm6813'), ('mm8699', 'dm5'), ('mm8699', 'dm490'), ('mm8699', 'dm713'), ('mm8699', 'dm5293'), ('mm8699', 'dm1301'), ('mm8699', 'dm7'), ('mm8699', 'dm7695'), ('mm8699', 'dm6283'), ('mm8699', 'dm8276'), ('mm8699', 'dm3544'), ('mm8699', 'dm2252'), ('mm8699', 'dm4241'), ('mm8699', 'dm2978'), ('mm8699', 'dm8419'), ('mm8699', 'dm7925'), ('mm8699', 'dm3241'), ('mm8699', 'dm2578'), ('mm8699'

# Compute Score for create similarity matrix 

In [27]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [28]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [29]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [30]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node
    

In [31]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [32]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [33]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=G_target_na.get_edge_data(str(nd1[i]),str(nd2[j]))
            if q1==None:
                c=0
            else:
                c=list(q1.values())
                c=c[0]
            seq[i][j]=c    
            
    return seq

In [34]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s

In [35]:
import numpy as np
m=compute_score(G1,G2)
n=m


# multi tasking for initial population

In [36]:
def task1(len_G1, similarity_score_matrix,m,index_score,pop,rand_permutation):
    print('task1')
    for z1 in range(int(len_G1/3)):
        for z2 in range(len_G1):
            similarity_score_matrix[z1][z2]=m[z1][int(rand_permutation[z1][z2])]
        index_score=np.argmax(similarity_score_matrix,axis=1)               
        pop[i][z1][0]=nd1.index(nd1[z1])
        pop[i][z1][1]=nd2.index(nd2[int(rand_permutation[z1][index_score[z1]])])
  

In [37]:
def task2(len_G1, similarity_score_matrix,m,index_score,pop,rand_permutation):
    print('task2')
    for z1 in range(int(len_G1/3),int(2*len_G1/3)):
        for z2 in range(len_G1):
            similarity_score_matrix[z1][z2]=m[z1][int(rand_permutation[z1][z2])]
        index_score=np.argmax(similarity_score_matrix,axis=1)               
        pop[i][z1][0]=nd1.index(nd1[z1])
        pop[i][z1][1]=nd2.index(nd2[int(rand_permutation[z1][index_score[z1]])])
    

In [38]:
def task3(len_G1, similarity_score_matrix,m,index_score,pop,rand_permutation):
    print('task3')
    for z1 in range(int(2*len_G1/3),len_G1):
        for z2 in range(len_G1):
            similarity_score_matrix[z1][z2]=m[z1][int(rand_permutation[z1][z2])]
        index_score=np.argmax(similarity_score_matrix,axis=1)               
        pop[i][z1][0]=nd1.index(nd1[z1])
        pop[i][z1][1]=nd2.index(nd2[int(rand_permutation[z1][index_score[z1]])])
    

# multi processing for speed up

In [39]:
def fun_rand_permute(len_G1, base_permute, rand_permutation):
    for k in range(len_G1):
        rand_permutation[k]=random.sample(base_permute,len_G1)
    #print(rand_permutation)

# Initialization such as MeAlign

In [40]:
#%%prun
import random
import threading

len_pop=300 
len_G1=len(G1)
len_G2=len(G2)
similarity_score_matrix=np.zeros((len_G1,len_G1))
sim_score_matrix=np.zeros((len_G1,len_G1))
pop=np.zeros((len_pop,len_G1,2))
index_score=np.zeros(len_G1)
rand_permutation=np.zeros((len_G1,len_G1))
base_permute = [j for  j in range(0, len_G2)]

for i in range(len_pop):  
    print('wait')
    if i>0:
        do_not_permute = set(base_permute)-set(index_score)#[item for item in base_permute if item not in index_score]
        #print(do_not_permute)
        #print(len(base_permute))
        base_permute.extend(do_not_permute)
        #print(len(do_not_permute))
        #print(len(base_permute))
        #print(base_permute)
     
    t = threading.Thread(target=fun_rand_permute, args=(len_G1,base_permute, rand_permutation,))
    t.start() 
    t.join()
    #print(rand_permutation)
    
    """for k in range(len_G1):
        rand_permutation[k]=random.sample(base_permute,len_G1) """               
    print('in for loop')
    """for z1 in range(len_G1):
        for z2 in range(len_G1):
            similarity_score_matrix[z1][z2]=m[z1][int(rand_permutation[z1][z2])]
        index_score=np.argmax(similarity_score_matrix,axis=1)               
        pop[i][z1][0]=nd1.index(nd1[z1])
        pop[i][z1][1]=nd2.index(nd2[int(rand_permutation[z1][index_score[z1]])])"""
     
    
      
    t1 = threading.Thread(target=task1, args=(len_G1, similarity_score_matrix,m,index_score,pop,rand_permutation,)) 
    t2 = threading.Thread(target=task2, args=(len_G1, similarity_score_matrix,m,index_score,pop,rand_permutation,))
    t3 = threading.Thread(target=task3, args=(len_G1, similarity_score_matrix,m,index_score,pop,rand_permutation,))
    # starting thread 1 
    t1.start() 
    # starting thread 2 
    t2.start() 
    # starting thread 3 
    t3.start() 
    # wait until thread 1 is completely executed 
    t1.join() 
    # wait until thread 2 is completely executed 
    t2.join() 
    # wait until thread 3 is completely executed 
    t3.join()
    # both threads completely executed 
    print("Done!, i=",i) 
    
pop

wait
in for loop
task1
task2
task3
Done!, i= 0
wait
in for loop
task1
task2
task3
Done!, i= 1
wait
in for loop
task1
task2
task3
Done!, i= 2
wait
in for loop
task1
task2
task3
Done!, i= 3
wait
in for loop
task1
task2
task3
Done!, i= 4
wait
in for loop
task1
task2
task3
Done!, i= 5
wait
in for loop
task1
task2task3

Done!, i= 6
wait
in for loop
task1
task2
task3
Done!, i= 7
wait
in for loop
task1
task2
task3
Done!, i= 8
wait
in for loop
task1
task2
task3
Done!, i= 9
wait
in for loop
task1
task2
task3
Done!, i= 10
wait
in for loop
task1task2

task3
Done!, i= 11
wait
in for loop
task1
task2task3

Done!, i= 12
wait
in for loop
task1
task2
task3
Done!, i= 13
wait
in for loop
task1task2

task3
Done!, i= 14
wait
in for loop
task1
task2
task3
Done!, i= 15
wait
in for loop
task1
task2
task3
Done!, i= 16
wait
in for loop
task1
task2
task3
Done!, i= 17
wait
in for loop
task1
task2
task3
Done!, i= 18
wait
in for loop
task1
task2
task3
Done!, i= 19
wait
in for loop
task1
task2task3

Done!, i= 20
wa

Done!, i= 169
wait
in for loop
task1
task2
task3
Done!, i= 170
wait
in for loop
task1
task2
task3
Done!, i= 171
wait
in for loop
task1
task2
task3
Done!, i= 172
wait
in for loop
task1
task2
task3
Done!, i= 173
wait
in for loop
task1
task2
task3
Done!, i= 174
wait
in for loop
task1
task2
task3
Done!, i= 175
wait
in for loop
task1
task2
task3
Done!, i= 176
wait
in for loop
task1
task2
task3
Done!, i= 177
wait
in for loop
task1
task2
task3
Done!, i= 178
wait
in for loop
task1
task2
task3
Done!, i= 179
wait
in for loop
task1
task2
task3
Done!, i= 180
wait
in for loop
task1
task2
task3
Done!, i= 181
wait
in for loop
task1
task2
task3
Done!, i= 182
wait
in for loop
task1
task2
task3
Done!, i= 183
wait
in for loop
task1
task2
task3
Done!, i= 184
wait
in for loop
task1
task2
task3
Done!, i= 185
wait
in for loop
task1
task2
task3
Done!, i= 186
wait
in for loop
task1
task2
task3
Done!, i= 187
wait
in for loop
task1
task2
task3
Done!, i= 188
wait
in for loop
task1
task2
task3
Done!, i= 189
wait
i

array([[[0.000e+00, 4.481e+03],
        [1.000e+00, 1.737e+03],
        [2.000e+00, 4.579e+03],
        ...,
        [6.200e+02, 3.270e+03],
        [6.210e+02, 7.069e+03],
        [6.220e+02, 4.554e+03]],

       [[0.000e+00, 7.147e+03],
        [1.000e+00, 3.326e+03],
        [2.000e+00, 4.102e+03],
        ...,
        [6.200e+02, 4.303e+03],
        [6.210e+02, 6.709e+03],
        [6.220e+02, 6.780e+03]],

       [[0.000e+00, 8.860e+02],
        [1.000e+00, 7.134e+03],
        [2.000e+00, 3.407e+03],
        ...,
        [6.200e+02, 4.303e+03],
        [6.210e+02, 3.412e+03],
        [6.220e+02, 6.896e+03]],

       ...,

       [[0.000e+00, 6.396e+03],
        [1.000e+00, 6.020e+03],
        [2.000e+00, 7.074e+03],
        ...,
        [6.200e+02, 2.320e+03],
        [6.210e+02, 5.646e+03],
        [6.220e+02, 4.555e+03]],

       [[0.000e+00, 4.983e+03],
        [1.000e+00, 3.758e+03],
        [2.000e+00, 5.718e+03],
        ...,
        [6.200e+02, 5.959e+03],
        [6.210e+02

# Save population

In [33]:
import pickle
with open('mm-dm-base_permute.pickle', 'wb') as f:
    pickle.dump(base_permute, f)

In [34]:
import pickle
with open('mm-dm-pop.pickle', 'wb') as f:
    pickle.dump(pop, f)

## You can load population if there is save population such as 'mm-dm-pop.pickle'

# Load population

In [ ]:
with open('mm-dm-base_permute', 'rb') as f:
    base_permute = pickle.load(f)

In [ ]:
with open('mm-dm-pop.pickle', 'rb') as f:
    pop = pickle.load(f)

# compute measure edge correctness, ics, nc and s3

In [41]:
def induced_function_for_initialpop(sub_pop):
    num_induced=0
    sub_g2=list(set([sub_pop[i][1] for i in range(len(G1))]))
    for i in range(len(sub_g2)):
        for j in range(i,len(sub_g2)):
            if G2.has_edge(nd2[int(sub_g2[i])], nd2[int(sub_g2[j])]):
                num_induced=num_induced+1
    return num_induced

In [42]:
score_ec=np.zeros(len_pop)
score_ics=np.zeros(len_pop)
score_s3=np.zeros(len_pop)
score_nc=np.zeros(len_pop)
num_seq=np.zeros(len_pop)
score_ncorectness=np.zeros(len_pop)

p=pop 
len_G1=len(G1)
map_items=[] #np.zeros((len_pop,len_G1,2))

for i in range(len(p)):
    ec=0
    nc=0
    sub_pop=p[i]
    sub_g2=list(set([sub_pop[i][1] for i in range(len(G1))]))
    num_induced=induced_function_for_initialpop(sub_pop) 
    #print("number induced for i=",i," is: ",num_induced)
    
    for j in range(len_G1):                           
         if G_target_na.has_edge(nd1[int(p[i][j][0])],nd2[int(p[i][j][1])]):
            num_seq[i]+=1  # create sequence matrix 
            map_items.append([nd1[int(p[i][j][0])],nd2[int(p[i][j][1])]])
            ec=ec+1            
            nc=nc+2
                 
    score_ec[i]=ec/G1.number_of_edges()*100
    score_ics[i]=ec/num_induced*100
    score_s3[i]=ec/(G1.number_of_edges()+num_induced-ec)*100
    score_nc[i]=nc/G1.number_of_nodes()*100 
    score_ncorectness[i]=(nc/G_target_na.number_of_nodes())*100
    """if (score_nc[i] > 100):
        score_nc[i]=100
    if (score_ec[i] > 100):
        score_ec[i]=100
    if (score_ics[i] > 100):
        score_ics[i]=100
    if (score_s3[i] > 100):
        score_s3[i]=100
    if ( score_ncorectness[i]>100):
         score_ncorectness[i]=100"""

print('ec')
print(score_ec)
print('ics')
print(score_ics)
print('s3')
print(score_s3)
print('nc',score_nc)
print('node correctness: ',  score_ncorectness)

ec
[14.31818182 14.20454545 15.         14.20454545 14.88636364 14.20454545
 14.82954545 14.43181818 13.69318182 13.57954545 14.88636364 14.77272727
 13.75       13.97727273 14.31818182 13.40909091 13.63636364 14.88636364
 13.69318182 14.09090909 14.43181818 14.375      14.26136364 13.125
 13.63636364 13.92045455 14.48863636 14.31818182 13.92045455 15.05681818
 14.48863636 13.86363636 13.52272727 13.75       15.85227273 14.77272727
 14.60227273 14.26136364 14.26136364 13.97727273 14.60227273 14.26136364
 13.75       14.14772727 14.20454545 15.90909091 14.60227273 14.20454545
 13.92045455 14.43181818 13.97727273 14.26136364 13.86363636 13.86363636
 14.14772727 14.43181818 14.03409091 14.31818182 14.20454545 14.77272727
 15.17045455 14.14772727 15.11363636 14.71590909 14.26136364 14.88636364
 13.86363636 13.69318182 13.63636364 14.60227273 14.65909091 13.35227273
 14.20454545 14.60227273 14.65909091 13.06818182 13.75       13.23863636
 13.92045455 14.48863636 13.69318182 13.80681818 14.3

## number of matched sequences 

In [43]:
num_seq

array([252., 250., 264., 250., 262., 250., 261., 254., 241., 239., 262.,
       260., 242., 246., 252., 236., 240., 262., 241., 248., 254., 253.,
       251., 231., 240., 245., 255., 252., 245., 265., 255., 244., 238.,
       242., 279., 260., 257., 251., 251., 246., 257., 251., 242., 249.,
       250., 280., 257., 250., 245., 254., 246., 251., 244., 244., 249.,
       254., 247., 252., 250., 260., 267., 249., 266., 259., 251., 262.,
       244., 241., 240., 257., 258., 235., 250., 257., 258., 230., 242.,
       233., 245., 255., 241., 243., 252., 250., 240., 244., 246., 251.,
       257., 239., 259., 242., 239., 253., 258., 254., 246., 254., 241.,
       255., 248., 259., 255., 265., 252., 263., 252., 260., 234., 258.,
       255., 255., 248., 254., 252., 246., 245., 240., 246., 246., 246.,
       262., 235., 250., 252., 250., 249., 252., 260., 244., 248., 260.,
       239., 236., 247., 236., 246., 245., 250., 262., 244., 261., 247.,
       254., 249., 262., 260., 248., 261., 258., 24

## compute topology score

In [44]:
#compute Final Score
tscore=np.zeros(len_pop)
for i in range(len(p)):
    tscore[i]=(0.25*(score_ec[i]/G1.number_of_edges())+0.25*(score_s3[i]/(G1.number_of_edges()+score_ics[i]-score_ec[i]))+0.25*(score_ics[i]/2*G1.number_of_edges())+0.25*(score_nc[i]/(G1.number_of_nodes()+G2.number_of_nodes())))
print(tscore)

[4389.55454824 4761.91061955 5194.99726752 4741.38516557 4596.49732401
 4680.85691186 4874.36943693 4633.50509942 4248.40303678 4463.50303776
 4744.03903661 4778.61927562 4036.39679973 4417.96491328 4379.15277273
 4807.41296564 4710.08592013 4759.70884178 4850.87483948 4631.58474627
 4800.6932341  5009.9069414  4683.63606506 4321.43396639 4275.3092266
 4537.04275978 5026.88771723 4386.08180357 4518.02751424 5078.40343288
 4678.90503834 4611.68955886 4374.27455799 4436.67231255 5277.7365569
 4931.04057714 4824.23809856 4389.51298745 4202.44112039 4681.66666123
 5289.06126115 4961.37158062 4678.38880279 4396.47449111 4625.74175392
 5012.21158164 4596.75395606 4610.2321597  5042.10119635 4660.55639066
 4295.2438092  4703.58338515 4533.78948456 4655.68654783 4490.16973756
 4591.62465799 4856.12734042 4647.11401791 4790.94662759 4475.7493862
 5388.99712508 4439.22783886 4659.24185875 4497.24358771 4507.76095038
 5397.00993598 4580.21048449 4248.40303678 4505.12506491 5043.71701462
 4671.610

# Compute Topology and Sequence score

In [45]:
# Final Score
score=np.zeros(len_pop)
alpha=0.5
for i in range (len(p)):
    score[i]=alpha*tscore[i]+(1-alpha)*num_seq[i]   
score

array([2320.77727412, 2505.95530977, 2729.49863376, 2495.69258279,
       2429.24866201, 2465.42845593, 2567.68471847, 2443.75254971,
       2244.70151839, 2351.25151888, 2503.0195183 , 2519.30963781,
       2139.19839986, 2331.98245664, 2315.57638636, 2521.70648282,
       2475.04296007, 2510.85442089, 2545.93741974, 2439.79237313,
       2527.34661705, 2631.4534707 , 2467.31803253, 2276.2169832 ,
       2257.6546133 , 2391.02137989, 2640.94385861, 2319.04090178,
       2381.51375712, 2671.70171644, 2466.95251917, 2427.84477943,
       2306.137279  , 2339.33615627, 2778.36827845, 2595.52028857,
       2540.61904928, 2320.25649372, 2226.72056019, 2463.83333062,
       2773.03063058, 2606.18579031, 2460.1944014 , 2322.73724556,
       2437.87087696, 2646.10579082, 2426.87697803, 2430.11607985,
       2643.55059817, 2457.27819533, 2270.6219046 , 2477.29169258,
       2388.89474228, 2449.84327392, 2369.58486878, 2422.812329  ,
       2551.56367021, 2449.55700896, 2520.47331379, 2367.87469

## Define function for fitness and compute final score

In [46]:
def induced_function_for_pop(wpop):
    counter_induced=0
    subg=list(set([wpop[i][1] for i in range(len_G1)]))
    for i in range(len(subg)):
        for j in range(i,len(subg)):
            if G2.has_edge(nd2[int(subg[i])], nd2[int(subg[j])]):
                counter_induced=counter_induced+1
    #print("con: ",counter_induced)
    return counter_induced

In [47]:
def Fitness(wpop,counter_induced):
    ec_temp=0
    nc_temp=0
    num_seq_temp=0
    score_ncorectness=0
    
  
    for j in range(len_G1):        
        x1=int(wpop[j][0])
        x2=int(wpop[j][1])
        if G_target_na.has_edge(nd1[x1],nd2[x2]):
            num_seq_temp+=1              
            ec_temp=ec_temp+1            
            nc_temp=nc_temp+2
                 
    score_ec_temp=ec_temp/G1.number_of_edges()*100
    score_ics_temp=ec_temp/counter_induced*100
    score_s3_temp=ec_temp/(G1.number_of_edges()+counter_induced-ec_temp)*100
    score_nc_temp=nc_temp/G1.number_of_nodes()*100
    score_ncorectness=(nc_temp/G_target_na.number_of_nodes())*100
    """if (score_nc_temp > 100):
        score_nc_temp=100
    if (score_ec_temp > 100):
        score_ec_temp=100
    if (score_ics_temp > 100):
        score_ics_temp=100
    if (score_s3_temp > 100):
        score_s3_temp=100
    if ( score_ncorectness>100):
         score_ncorectness=100"""
    # local measures
    pre=ec_temp/G_target_na.number_of_edges()*100
    rec=ec_temp/len(wpop)*100
    fsc=2*((pre*rec)/(pre+rec))
    
    if (GbestFlag==True):
        print('ec')
        print(score_ec_temp)
        print('ics')
        print(score_ics_temp)
        print('s3')
        print(score_s3_temp)
        print('nc',score_nc_temp)
        print('num_seq',num_seq_temp)
        print('precision: ',pre)
        print('Recall: ',rec)
        print('F-score: ',fsc)

    alpha=0.5
    tscore_temp=(0.25*(score_ec_temp/G1.number_of_edges())+0.25*(score_s3_temp/(G1.number_of_edges()+score_ics_temp-score_ec_temp))+0.25*(score_ics_temp/2*G1.number_of_edges())+0.25*(score_nc_temp/(G1.number_of_nodes()+G2.number_of_nodes())))
    score_temp=alpha*tscore_temp+(1-alpha)*num_seq_temp
    print('score:',score_temp)
    
    return score_temp,pre,rec,fsc


# HNA: Hybrid SFLA-CSO proposed method

In [48]:
# Implementation of Hybrid SFLA-CSO Algorithm
import random
from operator import itemgetter
from random import shuffle
import concurrent.futures

precision=np.zeros(len_pop)
recall=np.zeros(len_pop)
fscore=np.zeros(len_pop)

dim=len_G1
new_pop=p.tolist()
[new_pop[i].append(score[i]) for i in range(len_pop)]
#[print(new_pop[i][dim]) for i in range(len_pop)]

new_pop=sorted(new_pop, key=itemgetter(dim),reverse=True)
counter_induced=num_induced #0

GBestFrog=new_pop[0]
#print("GBest: ",GBestFrog)

index=0
iteration=2
epoch=0
memeplex=100

group=int(len_pop/memeplex)
len_G1=len(G1)
dim=len_G1
GbestFlag=False
max_score=max(score)
while (epoch<5):
    k=u=0
    #==== Set memeplexes =====
    memp=[]
    memp.append([new_pop[0],new_pop[memeplex],new_pop[2*memeplex]])#,new_pop[3*memeplex],new_pop[4*memeplex]])
    for i in range(1,memeplex):    
        memp.append([new_pop[i],new_pop[i+memeplex],new_pop[i+2*memeplex]])#,new_pop[i+3*memeplex],new_pop[i+4*memeplex]])
    #print('Check memeplex')
    #[[print(memp[i][j][dim]) for j in range(group)]for i in range(memeplex)]
    #=== Local Search =====   
    while(k<memeplex):
        u=0
        print("k ",k)
        print("epoch ",epoch)
        while(u<iteration):
            print("u ",u)
            memp[k]=sorted(memp[k], key=itemgetter(dim),reverse=True)
            #print('Check sort memeplex')
            #[[print(memp[i][j][dim]) for j in range(group)]for i in range(memeplex)]
            #=== set best frog and worst frog 
            bestFrog=memp[k][0]
            worstFrog=memp[k][group-1]            
            #print("BestFrog",bestFrog[dim])
            #print("WorstFrog",worstFrog[dim])
            r1=random.random()
            c1=2
            TeV=(worstFrog[dim]+r1*c1*(bestFrog[dim]-worstFrog[dim]))/100             #number of permutation
           # print(TeV)
                       
            # move 1
            temp_worst=worstFrog    #temp_worst is TeX
            Flag_move1=False
            # counter induced for ics compute            
            """with concurrent.futures.ThreadPoolExecutor() as executor:
                future = executor.submit(induced_function_for_pop, temp_worst)
                counter_induced = future.result()
                #print("counter induced: ",counter_induced)"""
            
            for i in range(int(TeV)):
                #Flag_move1=False
                rand_index1=random.randint(0,dim-1)
                rand_permute=random.sample(base_permute,len_G1)                
                #print('rand1',rand_index1)                                
                if not(G_target_na.has_edge(temp_worst[rand_index1][0],temp_worst[rand_index1][1])):                                                 
                        temp_score=np.argmax(m[rand_index1])
                        #print(nd2[temp_score])                        
                        #print('temp:',G_target_na.has_edge(nd1[int(temp_worst[rand_index1][0])],nd2[temp_score]))
                        if(G_target_na.has_edge(nd1[int(temp_worst[rand_index1][0])],nd2[temp_score])):                                
                                temp_worst[rand_index1][1]=temp_score
                                Fit_New,pre,rec,fsc=Fitness(temp_worst,counter_induced)     
                                if(Fit_New>worstFrog[dim]):
                                    print('w',worstFrog[dim])
                                    print('new',Fit_New)
                                    temp_worst[dim]=Fit_New
                                    memp[k][group-1]=temp_worst 
                                    Flag_move1=True
                                    print('move1 ', TeV)                                                                 
            #======== move 2 =======
            Flag_move2=False
            
            if(Flag_move1==False):
                    TeV=(worstFrog[dim]+r1*c1*(GBestFrog[dim]-worstFrog[dim]))/100   #number of random permutation                
                            #print(TeV)                
           
                    temp_worst=worstFrog    #temp_worst is TeX               
                
                    for i in range(int(TeV)):
                        #Flag_move2=False
                        rand_permute=random.sample(base_permute,len_G1)
                        rand_index1=random.randint(0,dim-1)
                        if not(G_target_na.has_edge(temp_worst[rand_index1][0],temp_worst[rand_index1][1])):                            
                            temp_score=np.argmax(m[rand_index1])
                            #print(nd2[temp_score])
                            #print('temp:',G_target_na.has_edge(nd1[int(temp_worst[rand_index1][0])],nd2[temp_score]))
                            if(G_target_na.has_edge(temp_worst[rand_index1][0],nd2[temp_score])):                                
                                temp_worst[rand_index1][1]=temp_score                                                                   
                                Fit_New,pre,rec,fsc=Fitness(temp_worst,counter_induced)     
                                if(Fit_New>worstFrog[dim]):
                                    print('w',worstFrog[dim])
                                    print('new',Fit_New)
                                    temp_worst[dim]=Fit_New
                                    memp[k][group-1]=temp_worst 
                                    Flag_move2=True
                                    print('move2 ', TeV)                                                                      
                    #===== move 3 =====
            
            if(Flag_move1==False and Flag_move2==False):
                TeV=(worstFrog[dim]*random.random()*2)/100             #number of permutation
                                            # print(TeV)
                              
                temp_worst=worstFrog   #temp_worst is TeX                    
                for i in range(int(TeV)):
                    rand_permute=random.sample(base_permute,len_G1)
                    rand_index1=random.randint(0,dim-1) 
                    if not(G_target_na.has_edge(temp_worst[rand_index1][0],temp_worst[rand_index1][1])):                        
                        temp_score=np.argmax(m[rand_index1])
                        #print(nd2[temp_score])
                        #print('temp:',G_target_na.has_edge(nd1[int(temp_worst[rand_index1][0])],nd2[temp_score]))
                        if (G_target_na.has_edge(temp_worst[rand_index1][0],nd2[temp_score])):                                                    
                            temp_worst[rand_index1][1]=temp_score                            
                            Fit_New,pre,rec,fsc=Fitness(temp_worst,counter_induced)     
                            if(Fit_New>worstFrog[dim]):
                                print('w',worstFrog[dim])
                                print('new',Fit_New)
                                temp_worst[dim]=Fit_New
                                memp[k][group-1]=temp_worst  
                                print('move3 ',TeV)
                
            u=u+1
        k=k+1    
    epoch+=1
    #==== combine memeplexes and create new pop ====
    print('*********** fitness **************')    
    new_pop=[]
    for i in range(memeplex):
        for j in range(group):        
            del memp[i][j][dim]
       
    [[new_pop.append(memp[i][j]) for j in range(group)] for i in range(memeplex)]         
    
    for i in range(len_pop):
        Fit,pre,rec,fsc=Fitness(new_pop[i],counter_induced)
        print('Fit pop',Fit)
        new_pop[i].append(Fit)
        score[i]=Fit
        precision[i]=pre
        recall[i]=rec
        fscore[i]=fsc
    
    new_pop=sorted(new_pop, key=itemgetter(dim), reverse=True)
    print('********** new population ************')
    [print('Fitness: ',new_pop[i][dim]) for i in range(len_pop)]
    
    GBestFrog=new_pop[0]
    
#print("GBest: ",GBestFrog)    
print("epoch: ",epoch)
print("score:", score)
#********** print Local Measures *********
print("Precision: ",precision)
print("Recall: ", recall)
print("F-measure: ",fscore)
#******** map items **********
map_items=[]
weight=[]
for i in range(len_pop):
    for j in range(len_G1):
        if G_target_na.has_edge(nd1[int(new_pop[i][j][0])],nd2[int(new_pop[i][j][1])]):
            map_items.append([nd1[int(new_pop[i][j][0])],nd2[int(new_pop[i][j][1])]]) 
            weight.append(G_target_na.get_edge_data(nd1[int(new_pop[i][j][0])],nd2[int(new_pop[i][j][1])])) 

print("end")

k  0
epoch  0
u  0
score: 2533.549056139447
w 2391.184481210859
new 2533.549056139447
move1  26.402876124430136
score: 2533.549056139447
score: 2543.523658965096
w 2533.549056139447
new 2543.523658965096
move1  26.402876124430136
score: 2543.523658965096
score: 2543.523658965096
score: 2553.4982617928285
w 2543.523658965096
new 2553.4982617928285
move1  26.402876124430136
score: 2563.472864622649
w 2553.4982617928285
new 2563.472864622649
move1  26.402876124430136
score: 2563.472864622649
score: 2573.447467454557
w 2563.472864622649
new 2573.447467454557
move1  26.402876124430136
score: 2583.4220702885573
w 2573.447467454557
new 2583.4220702885573
move1  26.402876124430136
score: 2593.396673124651
w 2583.4220702885573
new 2593.396673124651
move1  26.402876124430136
score: 2603.371275962842
w 2593.396673124651
new 2603.371275962842
move1  26.402876124430136
score: 2603.371275962842
score: 2613.34587880313
w 2603.371275962842
new 2613.34587880313
move1  26.402876124430136
score: 2613.345

score: 2433.8030279970876
w 2388.894742279267
new 2433.8030279970876
move1  24.46703007227083
score: 2433.8030279970876
score: 2443.7776308020207
w 2433.8030279970876
new 2443.7776308020207
move1  24.46703007227083
score: 2443.7776308020207
score: 2453.752233609015
w 2443.7776308020207
new 2453.752233609015
move1  24.46703007227083
score: 2463.7268364180736
w 2453.752233609015
new 2463.7268364180736
move1  24.46703007227083
score: 2463.7268364180736
score: 2463.7268364180736
score: 2463.7268364180736
score: 2473.7014392291967
w 2463.7268364180736
new 2473.7014392291967
move1  24.46703007227083
score: 2483.6760420423884
w 2473.7014392291967
new 2483.6760420423884
move1  24.46703007227083
score: 2493.6506448576506
w 2483.6760420423884
new 2493.6506448576506
move1  24.46703007227083
score: 2493.6506448576506
score: 2493.6506448576506
score: 2493.6506448576506
score: 2493.6506448576506
score: 2503.625247674984
w 2493.6506448576506
new 2503.625247674984
move1  24.46703007227083
score: 2503.

score: 2513.599850494394
score: 2513.599850494394
score: 2523.5744533158813
w 2513.599850494394
new 2523.5744533158813
move1  24.899480595216232
score: 2533.549056139447
w 2523.5744533158813
new 2533.549056139447
move1  24.899480595216232
score: 2533.549056139447
score: 2543.523658965096
w 2533.549056139447
new 2543.523658965096
move1  24.899480595216232
u  1
score: 2523.5744533158813
w 2493.2104637912316
new 2523.5744533158813
move1  29.31284114733255
score: 2533.549056139447
w 2523.5744533158813
new 2533.549056139447
move1  29.31284114733255
score: 2533.549056139447
score: 2543.523658965096
w 2533.549056139447
new 2543.523658965096
move1  29.31284114733255
score: 2543.523658965096
score: 2543.523658965096
score: 2543.523658965096
score: 2543.523658965096
score: 2553.4982617928285
w 2543.523658965096
new 2553.4982617928285
move1  29.31284114733255
score: 2563.472864622649
w 2553.4982617928285
new 2563.472864622649
move1  29.31284114733255
score: 2563.472864622649
score: 2563.472864622

score: 2593.396673124651
score: 2593.396673124651
score: 2593.396673124651
score: 2603.371275962842
w 2593.396673124651
new 2603.371275962842
move1  27.734484305725868
score: 2613.34587880313
w 2603.371275962842
new 2613.34587880313
move1  27.734484305725868
score: 2613.34587880313
u  1
score: 2463.7268364180736
score: 2463.7268364180736
score: 2473.7014392291967
score: 2473.7014392291967
score: 2473.7014392291967
score: 2473.7014392291967
score: 2483.6760420423884
score: 2483.6760420423884
score: 2493.6506448576506
w 2488.387499032166
new 2493.6506448576506
move1  28.07928781316456
score: 2493.6506448576506
score: 2503.625247674984
w 2493.6506448576506
new 2503.625247674984
move1  28.07928781316456
score: 2513.599850494394
w 2503.625247674984
new 2513.599850494394
move1  28.07928781316456
score: 2513.599850494394
score: 2523.5744533158813
w 2513.599850494394
new 2523.5744533158813
move1  28.07928781316456
score: 2533.549056139447
w 2523.5744533158813
new 2533.549056139447
move1  28.07

score: 2543.523658965096
score: 2553.4982617928285
w 2543.523658965096
new 2553.4982617928285
move1  26.63069343557717
k  14
epoch  0
u  0
score: 2533.549056139447
w 2378.4131091835307
new 2533.549056139447
move1  25.973031544369327
score: 2543.523658965096
w 2533.549056139447
new 2543.523658965096
move1  25.973031544369327
score: 2543.523658965096
score: 2553.4982617928285
w 2543.523658965096
new 2553.4982617928285
move1  25.973031544369327
score: 2563.472864622649
w 2553.4982617928285
new 2563.472864622649
move1  25.973031544369327
score: 2563.472864622649
score: 2573.447467454557
w 2563.472864622649
new 2573.447467454557
move1  25.973031544369327
score: 2573.447467454557
score: 2583.4220702885573
w 2573.447467454557
new 2583.4220702885573
move1  25.973031544369327
score: 2593.396673124651
w 2583.4220702885573
new 2593.396673124651
move1  25.973031544369327
score: 2603.371275962842
w 2593.396673124651
new 2603.371275962842
move1  25.973031544369327
score: 2613.34587880313
w 2603.3712

...

score: 2922.558567906044
w 2912.583964999507
new 2922.558567906044
move1  28.044325087090446
score: 2932.5331708147587
w 2922.558567906044
new 2932.5331708147587
move1  28.044325087090446
score: 2932.5331708147587
k  94
epoch  4
u  0
score: 2733.041113051147
score: 2733.041113051147
score: 2743.015715918934
w 2733.041113051147
new 2743.015715918934
move1  28.27989139358959
score: 2752.990318788852
w 2743.015715918934
new 2752.990318788852
move1  28.27989139358959
score: 2752.990318788852
score: 2762.9649216609055
w 2752.990318788852
new 2762.9649216609055
move1  28.27989139358959
score: 2772.9395245350966
w 2762.9649216609055
new 2772.9395245350966
move1  28.27989139358959
score: 2782.914127411426
w 2772.9395245350966
new 2782.914127411426
move1  28.27989139358959
score: 2792.8887302898975
w 2782.914127411426
new 2792.8887302898975
move1  28.27989139358959
score: 2802.8633331705128
w 2792.8887302898975
new 2802.8633331705128
move1  28.27989139358959
score: 2802.8633331705128
score: 280

score: 2812.8379360532754
w 2802.8633331705128
new 2812.8379360532754
move1  28.537791739833654
score: 2822.8125389381876
w 2812.8379360532754
new 2822.8125389381876
move1  28.537791739833654
score: 2832.78714182525
w 2822.8125389381876
new 2832.78714182525
move1  28.537791739833654
score: 2842.761744714467
w 2832.78714182525
new 2842.761744714467
move1  28.537791739833654
score: 2852.736347605841
w 2842.761744714467
new 2852.736347605841
move1  28.537791739833654
score: 2862.7109504993728
w 2852.736347605841
new 2862.7109504993728
move1  28.537791739833654
score: 2862.7109504993728
score: 2862.7109504993728
score: 2862.7109504993728
score: 2872.6855533950675
w 2862.7109504993728
new 2872.6855533950675
move1  28.537791739833654
score: 2872.6855533950675
score: 2872.6855533950675
score: 2872.6855533950675
score: 2872.6855533950675
score: 2882.6601562929254
w 2872.6855533950675
new 2882.6601562929254
move1  28.537791739833654
score: 2882.6601562929254
score: 2882.6601562929254
k  98
epoc

score: 2872.6855533950675
Fit pop 2872.6855533950675
score: 2902.6093620951424
Fit pop 2902.6093620951424
score: 2872.6855533950675
Fit pop 2872.6855533950675
score: 2832.78714182525
Fit pop 2832.78714182525
score: 2952.4823766387262
Fit pop 2952.4823766387262
score: 2902.6093620951424
Fit pop 2902.6093620951424
score: 2862.7109504993728
Fit pop 2862.7109504993728
score: 2912.583964999507
Fit pop 2912.583964999507
score: 2952.4823766387262
Fit pop 2952.4823766387262
score: 2862.7109504993728
Fit pop 2862.7109504993728
score: 2932.5331708147587
Fit pop 2932.5331708147587
score: 2892.6347591929493
Fit pop 2892.6347591929493
score: 2862.7109504993728
Fit pop 2862.7109504993728
score: 2962.456979553985
Fit pop 2962.456979553985
score: 2912.583964999507
Fit pop 2912.583964999507
score: 2862.7109504993728
Fit pop 2862.7109504993728
score: 2932.5331708147587
Fit pop 2932.5331708147587
score: 2932.5331708147587
Fit pop 2932.5331708147587
score: 2862.7109504993728
Fit pop 2862.7109504993728
sco

score: 2832.78714182525
Fit pop 2832.78714182525
score: 2912.583964999507
Fit pop 2912.583964999507
score: 2832.78714182525
Fit pop 2832.78714182525
score: 2822.8125389381876
Fit pop 2822.8125389381876
score: 2932.5331708147587
Fit pop 2932.5331708147587
score: 2852.736347605841
Fit pop 2852.736347605841
score: 2832.78714182525
Fit pop 2832.78714182525
score: 2842.761744714467
Fit pop 2842.761744714467
score: 2852.736347605841
Fit pop 2852.736347605841
score: 2822.8125389381876
Fit pop 2822.8125389381876
score: 2892.6347591929493
Fit pop 2892.6347591929493
score: 2862.7109504993728
Fit pop 2862.7109504993728
score: 2822.8125389381876
Fit pop 2822.8125389381876
score: 2882.6601562929254
Fit pop 2882.6601562929254
score: 2872.6855533950675
Fit pop 2872.6855533950675
score: 2822.8125389381876
Fit pop 2822.8125389381876
score: 2882.6601562929254
Fit pop 2882.6601562929254
score: 2872.6855533950675
Fit pop 2872.6855533950675
score: 2822.8125389381876
Fit pop 2822.8125389381876
score: 2892.6

end


In [49]:
# GbestFrog
gb=new_pop[0]
del gb[dim]
GbestFlag=True
fit_gb,pre,rec,fsc=Fitness(gb,counter_induced)
print('gbest fitness ', fit_gb)

ec
17.045454545454543
ics
25.839793281653744
s3
11.446012972148036
nc 96.30818619582665
num_seq 300
precision:  3.7674243375612204
Recall:  48.154093097913325
F-score:  6.988120195667364
score: 2992.3807883128866
gbest fitness  2992.3807883128866


# create weighted graph of network alignment

In [50]:
x=[[map_items[i][0],map_items[i][1],int(list(weight[i].values())[0])] for i in range(len(map_items))]
x

[['mm10249', 'dm5403', 60],
 ['mm4911', 'dm3701', 123],
 ['mm5453', 'dm1119', 77],
 ['mm9171', 'dm6375', 49],
 ['mm4668', 'dm12857', 151],
 ['mm3290', 'dm12857', 50],
 ['mm12315', 'dm7525', 100],
 ['mm5042', 'dm8465', 65],
 ['mm3362', 'dm2813', 736],
 ['mm9669', 'dm8297', 106],
 ['mm3314', 'dm2813', 655],
 ['mm8390', 'dm3027', 71],
 ['mm5138', 'dm3931', 400],
 ['mm9309', 'dm5525', 92],
 ['mm1793', 'dm6362', 62],
 ['mm2048', 'dm846', 70],
 ['mm3367', 'dm2623', 387],
 ['mm407', 'dm849', 71],
 ['mm8699', 'dm5', 238],
 ['mm14063', 'dm8667', 272],
 ['mm11243', 'dm7925', 186],
 ['mm2123', 'dm1286', 195],
 ['mm2153', 'dm576', 129],
 ['mm3821', 'dm2870', 454],
 ['mm7887', 'dm4845', 743],
 ['mm8967', 'dm4175', 141],
 ['mm2895', 'dm583', 242],
 ['mm1347', 'dm4531', 159],
 ['mm4294', 'dm4157', 256],
 ['mm11480', 'dm7935', 583],
 ['mm5183', 'dm583', 126],
 ['mm5034', 'dm1046', 221],
 ['mm6549', 'dm3529', 93],
 ['mm592', 'dm490', 154],
 ['mm5688', 'dm4157', 199],
 ['mm1150', 'dm7695', 119],
 ['mm14

## Remove duplication and local alignment
## Compute Global Alignment

In [51]:
unique_xmap=list(set(tuple(y) for y in x))

In [52]:
unique_xmap

[('mm11252', 'dm1330', 102),
 ('mm32', 'dm341', 64),
 ('mm1993', 'dm791', 54),
 ('mm2123', 'dm488', 171),
 ('mm2388', 'dm532', 85),
 ('mm1148', 'dm681', 88),
 ('mm2868', 'dm260', 73),
 ('mm11666', 'dm7122', 96),
 ('mm5394', 'dm720', 83),
 ('mm14063', 'dm4845', 89),
 ('mm2956', 'dm264', 69),
 ('mm765', 'dm1344', 154),
 ('mm1672', 'dm726', 428),
 ('mm2301', 'dm9069', 63),
 ('mm5610', 'dm5809', 65),
 ('mm115', 'dm98', 84),
 ('mm17603', 'dm10837', 69),
 ('mm1051', 'dm409', 60),
 ('mm12463', 'dm3581', 61),
 ('mm3056', 'dm2997', 49),
 ('mm7826', 'dm3906', 309),
 ('mm2999', 'dm5414', 51),
 ('mm9669', 'dm5241', 84),
 ('mm9669', 'dm7787', 173),
 ('mm2153', 'dm827', 165),
 ('mm2301', 'dm2978', 67),
 ('mm9407', 'dm6962', 103),
 ('mm603', 'dm286', 66),
 ('mm16941', 'dm8250', 64),
 ('mm271', 'dm163', 87),
 ('mm1150', 'dm7695', 119),
 ('mm303', 'dm7', 100),
 ('mm412', 'dm1048', 237),
 ('mm1150', 'dm1932', 214),
 ('mm7674', 'dm2516', 49),
 ('mm755', 'dm488', 193),
 ('mm32', 'dm1894', 194),
 ('mm10787

In [53]:
print(len(x))
print(len(unique_xmap))

86613
6600


# Compute Global Measures

In [54]:
len(map_items)

86613

In [55]:
unique_map=list(set(tuple(x) for x in map_items))
print(len(unique_map))

6600


In [56]:
# ec function
def ec_func(pop1,pop2):
    a=list(G1.neighbors(pop1));
    b=list(G2.neighbors(pop2));
    for i in range(len(a)):
        for j in range(len(b)):
           # print(pop1,a[i],G1.has_edge(pop1,a[i]))
           # print(pop2,b[j],G2.has_edge(pop2,b[j]))
           # print(G_target_na.has_edge(a[i],b[j]))
            if(G1.has_edge(pop1,a[i]) and G2.has_edge(pop2,b[j]) and G_target_na.has_edge(a[i],b[j])):
                return True
    return False 

In [57]:
def induced_function():
    count_induced=0
    subgraph=list(set([unique_map[i][1] for i in range(len(unique_map))]))
    for i in range(len(subgraph)):
        for j in range(i,len(subgraph)):
            if G2.has_edge(subgraph[i], subgraph[j]):
                count_induced=count_induced+1
    #print(count_induced)
    return count_induced

In [58]:
# Compute Global Measures
ec_temp=0
nc_temp=0
num_seq_temp=0
score_ncorrectness=0

count_induced=induced_function()

len_unique=len(unique_map)
for i in range(len_unique):        
    if G_target_na.has_edge(unique_map[i][0],unique_map[i][1]):
            num_seq_temp+=1              
            #ec_temp=ec_temp+1
            if (ec_func(unique_map[i][0],unique_map[i][1])):
                ec_temp=ec_temp+1                
            nc_temp=nc_temp+2
                 
score_ec_temp=ec_temp/G1.number_of_edges()*100
score_ics_temp=ec_temp/count_induced*100
score_s3_temp=ec_temp/(G1.number_of_edges()+count_induced-ec_temp)*100
score_nc_temp=nc_temp/G1.number_of_nodes()*100
score_ncorrectness=(nc_temp/G_target_na.number_of_nodes())*100
"""if (score_nc_temp > 100):
    score_nc_temp=100
if (score_ec_temp > 100):
    score_ec_temp=100
if (score_ics_temp > 100):
    score_ics_temp=100
if (score_s3_temp > 100):
    score_s3_temp=100
if ( score_ncorrectness>100):
    score_ncorrectness=100""" 

print('ec')
print(score_ec_temp)
print('ics')
print(score_ics_temp)
print('s3')
print(score_s3_temp)
print('nc',score_nc_temp)
print('num_seq',num_seq_temp)
print('node correctness: ',score_ncorrectness)

alpha=0.5
tscore_temp=(0.25*(score_ec_temp/G1.number_of_edges())+0.25*(score_s3_temp/(G1.number_of_edges()+score_ics_temp-score_ec_temp))+0.25*(score_ics_temp/2*G1.number_of_edges())+0.25*(score_nc_temp/(G1.number_of_nodes()+G2.number_of_nodes())))
score_temp=alpha*tscore_temp+(1-alpha)*num_seq_temp
print('score:',score_temp)    

ec
375.0
ics
34.602076124567475
s3
46.36785162287481
nc 2118.7800963081863
num_seq 6600
node correctness:  566.2805662805663
score: 7106.292262329922


# save and load map

In [54]:
import pickle
with open('mm-dm-weightGraph.pickle', 'wb') as f:
    pickle.dump(unique_xmap, f)

In [ ]:
import pickle
with open('mm-dm-weightGraph.pickle', 'rb') as f:
    unique_xmap = pickle.load(f)

In [59]:
import matplotlib.pyplot as plt

G_Align=nx.Graph()
elist = unique_xmap
G_Align.add_weighted_edges_from(elist)
nx.draw(G_Align)
plt.show()

## compare node and edge connectivity
## compute largest connected component

In [60]:
print('Aligned graph have ',G_Align.number_of_nodes(),' nodes.')
print('Aligned graph have ',G_Align.number_of_edges(),' edges.')

Aligned graph have  2257  nodes.
Aligned graph have  6600  edges.


In [61]:
print('Target graph have ',G_target_na.number_of_nodes(),' nodes.')
print('Target graph have ',G_target_na.number_of_edges(), ' edges.')

Target graph have  2331  nodes.
Target graph have  7963  edges.


In [62]:
sum_weight=0
for i in range(len(unique_xmap)):
    sum_weight=sum_weight+ unique_xmap[i][2]
total_weight=0    
for i in range(len(nd1)):
    for j in range(len(nd2)):
        temp=G_target_na.get_edge_data(str(nd1[i]),str(nd2[j]))
        if(temp!=None):
            total_weight=total_weight+int(list(temp.values())[0])
print('Maximum Weight in Aligned graph: ',sum_weight) 
print('Total Weight in Target: ',total_weight)

Maximum Weight in Aligned graph:  804845
Total Weight in Target:  899908


In [63]:
# returns number of different connected components 
print('number of different connected components in Aligned graph: ',nx.number_connected_components(G_Align)) 
print('number of different connected components in Target graph: ',nx.number_connected_components(G_target_na))  

number of different connected components in Aligned graph:  159
number of different connected components in Target graph:  155


In [64]:
# returns list of nodes in different connected components 
cc_align=list(nx.connected_components(G_Align))
cc_target=list(nx.connected_components(G_target_na))
print(list(nx.connected_components(G_Align))) 
print(list(nx.connected_components(G_target_na)))

[{'mm8533', 'mm8758', 'mm15591', 'dm5381', 'dm3071', 'mm8460', 'dm3091', 'dm2847', 'dm6076', 'dm543', 'mm8139', 'dm1474', 'mm8324', 'mm6512', 'mm8183', 'dm1330', 'dm3497', 'mm11252', 'dm2691', 'dm2068', 'mm8059', 'dm6465', 'mm3195', 'dm3972', 'dm5189', 'mm7654', 'mm5613', 'mm8008', 'dm2794', 'dm2382', 'dm7570'}, {'dm2744', 'dm4787', 'mm9696', 'mm11243', 'dm834', 'dm312', 'mm11761', 'dm1735', 'dm9665', 'dm2833', 'mm14346', 'dm130', 'dm3529', 'dm13977', 'dm1335', 'dm2816', 'mm18233', 'dm1105', 'dm841', 'dm91', 'dm2032', 'dm1382', 'dm5533', 'mm5034', 'dm3557', 'dm736', 'dm9314', 'dm2260', 'dm5739', 'dm253', 'dm80', 'dm7180', 'dm956', 'mm5695', 'dm2390', 'dm4120', 'dm1063', 'dm2017', 'dm6304', 'mm2606', 'dm2989', 'dm7524', 'dm985', 'mm3944', 'dm2999', 'dm1141', 'mm2827', 'dm6513', 'mm11184', 'dm7581', 'dm9577', 'dm115', 'dm10436', 'dm4442', 'dm2346', 'mm2479', 'dm3998', 'mm8804', 'mm15188', 'dm12443', 'dm6386', 'dm4845', 'dm1826', 'dm9313', 'dm1777', 'dm640', 'mm988', 'dm3663', 'dm532', 'd

In [65]:
# returns number of nodes to be removed 
# so that Graph becomes disconnected 
print(nx.node_connectivity(G_Align)) 
print(nx.node_connectivity(G_target_na))
# returns number of edges to be removed 
# so that Graph becomes disconnected 
print(nx.edge_connectivity(G_Align))
print(nx.edge_connectivity(G_target_na))

0
0
0
0


## check similarity of connected components

In [66]:
count=0
for i in range(len(cc_align)):
    for j in range(len(cc_target)):
        if(cc_align[i]==cc_target[j]):
            count=count+1
print(count)

154


In [67]:
Gc = max(nx.connected_component_subgraphs(G_Align), key=len)
print('Largest common connected graph in Aligned graph: ',len(Gc))
Gct = max(nx.connected_component_subgraphs(G_target_na), key=len)
print('Largest common connected graph in Target graph: ',len(Gct))

Largest common connected graph in Aligned graph:  1311
Largest common connected graph in Target graph:  1566


### check for mistake align

In [55]:
for i in range(len(unique_xmap)):
    if (G_target_na.has_edge(unique_xmap[i][0], unique_xmap[i][1])==False):
        print(unique_xmap[i])
        print(i)
print('no exist error')    

no exist error


# draw graphs

In [0]:
nx.draw_networkx(G1, with_labels = True)

In [0]:
nx.draw_networkx(G2)

In [0]:
import matplotlib.pyplot as plt
nx.draw(G_target_na)
plt.show()

In [68]:
# total ec
total_ec=G_target_na.number_of_edges()/G1.number_of_edges()*100
ratio_ec=score_ec_temp/total_ec*100
print("total EC: ",total_ec)
print("ratio EC: ",ratio_ec)

total EC:  452.4431818181818
ratio EC:  82.88333542634686


In [69]:
# total ics
total_ics=G_target_na.number_of_edges()/count_induced*100
ratio_ics=score_ics_temp/total_ics*100
print("total ICS: ",total_ics)
print("ratio ICS: ",ratio_ics)

total ICS:  41.747929118171335
ratio ICS:  82.88333542634685


In [71]:
# total s3
total_s3=(G_target_na.number_of_edges()/(G1.number_of_edges()+count_induced-G_target_na.number_of_edges()))*100
ratio_s3=score_s3_temp/total_s3*100
print("total S3: ",total_s3)
print("ratio S3: ",ratio_s3)

total S3:  61.86776474244425
ratio S3:  74.94670579405019


# compute FC

In [1]:
import pickle
with open('mm-dm-weightGraph.pickle', 'rb') as f:
    unique_xmap = pickle.load(f)

In [2]:
tree=[[unique_xmap[i][0],unique_xmap[i][1]] for i in range(len(unique_xmap))]
tree

[['mm6469', 'dm4652'],
 ['mm1225', 'dm281'],
 ['mm1082', 'dm2195'],
 ['mm3398', 'dm3939'],
 ['mm16941', 'dm11068'],
 ['mm11252', 'dm3497'],
 ['mm3362', 'dm4521'],
 ['mm6350', 'dm2346'],
 ['mm12833', 'dm9643'],
 ['mm1472', 'dm1407'],
 ['mm6374', 'dm488'],
 ['mm126', 'dm6259'],
 ['mm4686', 'dm3213'],
 ['mm2307', 'dm1032'],
 ['mm1226', 'dm2054'],
 ['mm16191', 'dm10730'],
 ['mm2545', 'dm1465'],
 ['mm1953', 'dm1361'],
 ['mm126', 'dm16'],
 ['mm1012', 'dm444'],
 ['mm271', 'dm2055'],
 ['mm1377', 'dm952'],
 ['mm1993', 'dm532'],
 ['mm7826', 'dm1932'],
 ['mm3290', 'dm12194'],
 ['mm6350', 'dm4989'],
 ['mm3821', 'dm1181'],
 ['mm765', 'dm4531'],
 ['mm508', 'dm5107'],
 ['mm14063', 'dm7'],
 ['mm12315', 'dm10273'],
 ['mm2301', 'dm2978'],
 ['mm5380', 'dm3742'],
 ['mm6608', 'dm2199'],
 ['mm5138', 'dm8646'],
 ['mm304', 'dm2572'],
 ['mm5919', 'dm792'],
 ['mm5925', 'dm8817'],
 ['mm592', 'dm4334'],
 ['mm8008', 'dm5189'],
 ['mm665', 'dm2833'],
 ['mm2388', 'dm227'],
 ['mm17071', 'dm10880'],
 ['mm17603', 'dm110

# load ontology and create DAG graph

In [3]:
import pandas as pd
f=open('go.dag.obo.txt').readlines()
df=pd.DataFrame(f)
df

,0
0,GO:0046431 GO:0019599\n
1,GO:0046430 GO:0019595\n
2,GO:0046432 GO:0019361\n
3,GO:0034198 GO:0060733\n
4,GO:0046343 GO:0019872\n
5,GO:0034193 GO:0034195\n
6,GO:0034192 GO:0034194\n
7,GO:0034190 GO:0034191\n
8,GO:0046437 GO:0030632 GO:0070179\n
9,GO:0046340 GO:0046342\n


In [4]:
# preprocessing
dag_list=[]
for i in range(len(df[0])):
    m1=str(df[0][i])
    m1=m1.replace('GO:','')
    m1=m1.replace('\n','')
    s=m1.split(' ')
    dag_list.append(s)
dag_list

[['0046431', '0019599'],
 ['0046430', '0019595'],
 ['0046432', '0019361'],
 ['0034198', '0060733'],
 ['0046343', '0019872'],
 ['0034193', '0034195'],
 ['0034192', '0034194'],
 ['0034190', '0034191'],
 ['0046437', '0030632', '0070179'],
 ['0046340', '0046342'],
 ['0030852', '0045658', '0030853', '0045643', '0030854', '0045640'],
 ['0030853', '0045659', '0045644', '0045641'],
 ['0030851', '0030221', '0030223', '0030222'],
 ['0030856',
  '0003339',
  '0030860',
  '0045601',
  '0030857',
  '0072182',
  '0030858',
  '0045616'],
 ['0030857', '0045602', '0030861', '0045617', '0072183'],
 ['0030854', '0045645', '0045642', '0045660'],
 ['0030855',
  '0030859',
  '0030216',
  '0072192',
  '0060671',
  '0072160',
  '0060742',
  '0035026',
  '0072311',
  '0060487',
  '0045446',
  '0060644',
  '0002065'],
 ['0030858', '0045603', '0030862', '0045618'],
 ['0030859', '0045198'],
 ['0031106', '0032186'],
 ['0031105', '0032151', '0032152'],
 ['0031103', '0014814', '0048673', '0014012'],
 ['0031102', '00

In [5]:
dag_nd=[]
for i in range(len(dag_list)):
    for j in range(len(dag_list[i])):
        dag_nd.append(dag_list[i][j])
dag_nodes=set(dag_nd)
print(dag_nodes)
print(len(dag_nodes))

{'0047518', '0034354', '0032363', '0035221', '0035302', '0050292', '0043225', '0030552', '0002453', '0010648', '0071573', '0010865', '0006150', '0002521', '0051018', '0043396', '0075015', '0015242', '0034935', '0072255', '0070771', '0042636', '0034857', '0003692', '0009961', '0005219', '0046567', '0055082', '0022026', '0021860', '0060283', '0047831', '0033363', '0008377', '0019388', '0051773', '0047082', '0015171', '0030866', '0035561', '0010687', '0035303', '0085025', '0032554', '0021990', '0031270', '0003088', '0009083', '0018786', '0047321', '0010638', '0051026', '0006812', '0003095', '0051240', '0032088', '0046622', '0030651', '0005157', '0008170', '0070872', '0046816', '0052245', '0047100', '0047591', '0034130', '0030900', '0031439', '0005895', '0043421', '0071497', '0047023', '0048308', '0046087', '0015726', '0048725', '0032143', '0043094', '0071687', '0048198', '0033922', '0052323', '0007628', '0048526', '0075165', '0035406', '0019105', '0010679', '0032207', '0047742', '0017099'

In [6]:
# create dag edges
dag_edges=[]
for i in range(len(dag_list)):
    for j in range(1,len(dag_list[i])):
        ed=[dag_list[i][0],dag_list[i][j]]
        dag_edges.append(ed)
dag_edges

[['0046431', '0019599'],
 ['0046430', '0019595'],
 ['0046432', '0019361'],
 ['0034198', '0060733'],
 ['0046343', '0019872'],
 ['0034193', '0034195'],
 ['0034192', '0034194'],
 ['0034190', '0034191'],
 ['0046437', '0030632'],
 ['0046437', '0070179'],
 ['0046340', '0046342'],
 ['0030852', '0045658'],
 ['0030852', '0030853'],
 ['0030852', '0045643'],
 ['0030852', '0030854'],
 ['0030852', '0045640'],
 ['0030853', '0045659'],
 ['0030853', '0045644'],
 ['0030853', '0045641'],
 ['0030851', '0030221'],
 ['0030851', '0030223'],
 ['0030851', '0030222'],
 ['0030856', '0003339'],
 ['0030856', '0030860'],
 ['0030856', '0045601'],
 ['0030856', '0030857'],
 ['0030856', '0072182'],
 ['0030856', '0030858'],
 ['0030856', '0045616'],
 ['0030857', '0045602'],
 ['0030857', '0030861'],
 ['0030857', '0045617'],
 ['0030857', '0072183'],
 ['0030854', '0045645'],
 ['0030854', '0045642'],
 ['0030854', '0045660'],
 ['0030855', '0030859'],
 ['0030855', '0030216'],
 ['0030855', '0072192'],
 ['0030855', '0060671'],


In [7]:
import networkx as nx
go=nx.DiGraph()
go.add_nodes_from(dag_nodes)
go.add_edges_from(dag_edges)
go

In [8]:
c=0
import numpy as np
na=tree
for i in range(len(tree)):
    for j in range(2):
        if ('dm' in tree[i][j] or 'mm' in tree[i][j]):
            s=tree[i][j][2:]
            x='0'*(7-len(s))
            y=x+s
            #print(y)
            na[i][j]=y
            c=c+1
print(len(tree),c)
print(len(na))

6627 13254
6627


In [9]:
fc=[]
for i in range(len(na)):
    if na[i][0] in dag_nodes:
        e=go.predecessors(na[i][0])
        if na[i][1] in dag_nodes: 
            f=go.predecessors(na[i][1])
            temp=[]
            for key in e:
                temp.append(key)
            for key in f:
                temp.append(key)
            # compute intersection/union 
            fc.append(sum([temp.count(temp[z])/len(temp) for z in range(len(temp))]))
        else:
            fc.append(0)
#print(fc)
# compute average of all fc 
func=np.average(fc)
print('functional coherence:',func)

functional coherence: 0.6263061775448626
